In [11]:
# Import required libraries
import pandas as pd  # pandas is used for data manipulation and analysis using dataframes
import matplotlib.pyplot as plt  # matplotlib is used for creating static plots
import seaborn as sns  # seaborn is used for statistical data visualization
import numpy as np  # numpy is used for numerical operations like mean, sum, etc.
from matplotlib.ticker import MaxNLocator, FuncFormatter  # used to format tick labels on axes (e.g., integers, percentages)
import plotly.express as px  # plotly.express is used for quick interactive visualizations
import plotly.graph_objects as go  # plotly.graph_objects is used for creating detailed interactive plots

# Load dataset from CSV file
data = pd.read_csv('/content/hospital_datas45.csv')  # load the CSV file into a pandas dataframe

# Convert admission and discharge columns to datetime objects for date calculations
data['admission_date'] = pd.to_datetime(data['admission_date'])  # convert admission date column to datetime format
data['discharge_date'] = pd.to_datetime(data['discharge_date'])  # convert discharge date column to datetime format
print(data.head())

  Doctor Name Doctor ID Patient Name Patient ID admission_date discharge_date  \
0         Sai      R152       Swathi       A118     2021-01-01     2021-01-14   
1      Rajesh      A789        Surya       O346     2021-01-02     2021-01-14   
2       Priya      G158       Edward       N542     2021-01-03     2021-01-14   
3        Ravi      F212       Justin       O465     2021-01-04     2021-01-16   
4      Mahesh      J545        Neetu       L654     2021-01-05     2021-01-17   

  Treatment  Bill Payment Patient Gender        ward  readmission_flag  
0    Cancer          1500         Female     Surgery               1.0  
1    Tyroid          2500           Male  Pediatrics               1.0  
2  Diabetes          4586           Male     General               0.0  
3     Covid          2647           Male     Surgery               1.0  
4        BP          2560         Female     Surgery               0.0  


In [12]:
# Calculate patient's length of stay in days
data['length_of_stay'] = (data['discharge_date'] - data['admission_date']).dt.days  # compute stay duration
print(data['length_of_stay'])

0      13
1      12
2      11
3      12
4      12
       ..
281     9
282     8
283     7
284     6
285     5
Name: length_of_stay, Length: 286, dtype: int64


In [13]:
# -----------------------------
# AVERAGE LENGTH OF STAY BY WARD
# -----------------------------
avg_stay = data.groupby('ward')['length_of_stay'].mean().reset_index()  # calculate average length of stay grouped by ward
print(avg_stay)

         ward  length_of_stay
0     General        4.913043
1         ICU        3.913043
2  Pediatrics        4.629630
3     Surgery        4.055556


In [14]:
# Line Chart – Avg Length of Stay
fig_stay = px.line(avg_stay, x='ward', y='length_of_stay', title='Average Length of Stay by Ward (Interactive)',
                   labels={'ward': 'Ward', 'length_of_stay': 'Avg Stay (Days)'}, markers=True, template='plotly_white')
fig_stay.update_traces(hovertemplate='<b>Ward</b>: %{x}<br><b>Avg Stay</b>: %{y:.1f} days', line=dict(color='darkgreen'))
fig_stay.update_layout(xaxis_title='Ward', yaxis_title='Average Stay (Days)', hovermode='x unified')
fig_stay.show()


In [15]:
# Bar Chart – Avg Length of Stay
fig_bar_stay = px.bar(avg_stay, x='ward', y='length_of_stay', title='Average Stay Metrics by Ward (Bar)',
                      labels={'ward': 'Ward', 'length_of_stay': 'Avg Stay (Days)'}, template='plotly_white',
                      color='length_of_stay', color_continuous_scale='Blues')
fig_bar_stay.update_traces(hovertemplate='<b>Ward</b>: %{x}<br><b>Avg Stay</b>: %{y:.1f} days')
fig_bar_stay.update_layout(xaxis_title='Ward', yaxis_title='Average Stay (Days)')
fig_bar_stay.show()

In [16]:
# -----------------------------
# BED OCCUPANCY RATE BY WARD
# -----------------------------
data['bed_days_used'] = data['length_of_stay']
bed_occupancy = data.groupby('ward')['bed_days_used'].sum().reset_index()
total_beds = data['ward'].value_counts().max()
avg_los = data['length_of_stay'].mean()
bed_occupancy['bed_occupancy_rate'] = bed_occupancy['bed_days_used'] / (total_beds * avg_los)


In [17]:
# Line Chart – Bed Occupancy Rate
fig_occ = px.line(bed_occupancy, x='ward', y='bed_occupancy_rate',
                  title='Bed Occupancy Rate by Ward (Interactive)', labels={'ward': 'Ward', 'bed_occupancy_rate': 'Occupancy Rate'},
                  markers=True, template='plotly_white')
fig_occ.update_traces(hovertemplate='<b>Ward</b>: %{x}<br><b>Occupancy Rate</b>: %{y:.1%}', line=dict(color='purple'))
fig_occ.update_layout(xaxis_title='Ward', yaxis_title='Occupancy Rate (%)', hovermode='x unified', yaxis_tickformat='.0%')
fig_occ.show()


In [18]:
# Bar Chart – Bed Occupancy Rate
fig_bar_occ = px.bar(bed_occupancy, x='ward', y='bed_occupancy_rate',
                     title='Bed Utilization Rate by Ward (Bar)', labels={'ward': 'Ward', 'bed_occupancy_rate': 'Occupancy Rate'},
                     template='plotly_white', color='bed_occupancy_rate', color_continuous_scale='Oranges')
fig_bar_occ.update_traces(hovertemplate='<b>Ward</b>: %{x}<br><b>Occupancy Rate</b>: %{y:.1%}')
fig_bar_occ.update_layout(xaxis_title='Ward', yaxis_title='Occupancy Rate (%)', yaxis_tickformat='.0%')
fig_bar_occ.show()


In [19]:
# -----------------------------
# DAILY BED OCCUPANCY TRACKING
# -----------------------------
records = []
for _, row in data.iterrows():
    stay_dates = pd.date_range(start=row['admission_date'], end=row['discharge_date'])
    for date in stay_dates:
        records.append({'date': date, 'ward': row['ward']})
daily_df = pd.DataFrame(records)
daily_occupancy = daily_df.groupby('date').size().reset_index(name='occupied_beds')
print(daily_occupancy)

         date  occupied_beds
0  2021-01-01             11
1  2021-01-02             22
2  2021-01-03             33
3  2021-01-04             40
4  2021-01-05             47
5  2021-01-06             54
6  2021-01-07             61
7  2021-01-08             68
8  2021-01-09             75
9  2021-01-10             82
10 2021-01-11             89
11 2021-01-12             95
12 2021-01-13            100
13 2021-01-14             71
14 2021-01-15             72
15 2021-01-16             61
16 2021-01-17             63
17 2021-01-18             54
18 2021-01-19             57
19 2021-01-20             60
20 2021-01-21             62
21 2021-01-22             49
22 2021-01-23             52
23 2021-01-24             55
24 2021-01-25             58
25 2021-01-26             53
26 2021-01-27             39
27 2021-01-28             30
28 2021-01-29             15
29 2021-01-30              7


In [20]:
fig_daily = px.line(daily_occupancy, x='date', y='occupied_beds',
                    title='Daily Bed Occupancy Over Time (Interactive)',
                    labels={'date': 'Date', 'occupied_beds': 'Number of Occupied Beds'},
                    markers=True, template='plotly_white')
fig_daily.update_traces(hovertemplate='<b>Date</b>: %{x|%Y-%m-%d}<br><b>Occupied Beds</b>: %{y}', line=dict(color='red'))
fig_daily.update_layout(xaxis_title='Date', yaxis_title='Occupied Beds', hovermode='x unified')
fig_daily.show()

In [23]:
# 2. Admission/Discharge Trend
admit_discharge = data.groupby('admission_date').size().reset_index(name='Admissions')
discharge_trend = data.groupby('discharge_date').size().reset_index(name='Discharges')

fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=admit_discharge['admission_date'], y=admit_discharge['Admissions'],
                               mode='lines+markers', name='Admissions', line=dict(color='green')))
fig_trend.add_trace(go.Scatter(x=discharge_trend['discharge_date'], y=discharge_trend['Discharges'],
                               mode='lines+markers', name='Discharges', line=dict(color='orange')))
fig_trend.update_layout(title='Admission & Discharge Trends', xaxis_title='Date', yaxis_title='Count',
                        template='plotly_white', hovermode='x unified')
fig_trend.show()

In [25]:
# 3. Day-wise Admission Rates
data['admit_day'] = data['admission_date'].dt.day_name()
admit_by_day = data['admit_day'].value_counts().reset_index()
admit_by_day.columns = ['Day', 'Admissions']
fig_daywise = px.bar(admit_by_day, x='Day', y='Admissions', title='Day-wise Admission Rates',
                     template='plotly_white', color='Admissions', color_continuous_scale='Viridis')
fig_daywise.update_traces(hovertemplate='<b>Day</b>: %{x}<br><b>Admissions</b>: %{y}')
fig_daywise.show()


In [26]:
# 4. Ward-Level Congestion – Patient Count
ward_congestion = data['ward'].value_counts().reset_index()
ward_congestion.columns = ['Ward', 'Patients']
fig_congestion = px.bar(ward_congestion, x='Ward', y='Patients', title='Ward-Level Congestion (Patient Volume)',
                        template='plotly_white', color='Patients', color_continuous_scale='Reds')
fig_congestion.update_traces(hovertemplate='<b>Ward</b>: %{x}<br><b>Patients</b>: %{y}')
fig_congestion.show()

In [27]:
# 5. Patient Volume Trend
volume_trend = data.groupby('admission_date').size().reset_index(name='Admissions')
fig_volume = px.line(volume_trend, x='admission_date', y='Admissions', title='Patient Volume Trend Over Time',
                     labels={'admission_date': 'Date'}, template='plotly_white', markers=True)
fig_volume.update_traces(hovertemplate='<b>Date</b>: %{x|%Y-%m-%d}<br><b>Admissions</b>: %{y}', line=dict(color='blue'))
fig_volume.update_layout(xaxis_title='Date', yaxis_title='Number of Admissions', hovermode='x unified')
fig_volume.show()